In [4]:
# importing the requests library
import requests
from cvxopt import matrix, solvers
import json
import datetime
from dateutil.relativedelta import *
from forex_python.converter import CurrencyRates
import numpy as np
import re
solvers.options['show_progress'] = False

In [5]:
import requests
import json
class RequestManadger:
    def __init__(self, url,  user = '', password = '',auth = True):
        self.url = url
        self.session = requests.Session()
        self.ids = []
        if (auth):
            if ((user != '') and (password != '')):
                self.user = user
                self.session.auth = (user, password)
                self.session.headers.update({'x-test': 'true'})
                r = self.session.get(self.url, verify = False)
                #self.port_id= rm.get_request("asset?columns=ASSET_DATABASE_ID&columns=LABEL&columns=TYPE&TYPE=PORTFOLIO")[0]['ASSET_DATABASE_ID']['value']
                self.port_id = 573
                self.load_datas()
                
            else :
                print("you should precise explicitly auth = false if "
                      "you dont want add a user name and a password")
        else:
            return self.session.get(self.url, verify = False)
    def load_datas(self):
        datas = {'ids':'my_ids.txt',"ass":"my_assets.txt",
                 "corr":"my_corr.txt",'ass65':"my_assets65.txt",
                 "var":"my_vars.txt","shp65":"my_sharpes65.txt",
                 "cov":"my_cov.txt","shp":"my_sharpes.txt",
                 "prx_cur":"my_prx_curr.txt"}
        
        self.ids = self.read_file(datas['ids']).astype(int)
        self.assets = self.read_file(datas['ass'])
        self.sharpes = self.read_file(datas['shp'])
        self.assets65 = self.read_file(datas['ass65'])
        self.sharpes65 = self.read_file(datas['shp65'])
        self.prx_cur = self.read_file(datas['prx_cur'], prx_cur = True).T
        self.vars = self.read_file(datas['var'])
        self.corr =  self.read_file(datas['corr'])
        self.cov =  self.read_file(datas['cov'])
        self.cov2 = np.cov(self.assets65)
        self.corr2 = np.array([[self.cov2[i][j]/np.sqrt(np.var(self.assets65[i]) * np.var(self.assets65[j])) for j in range(self.cov2.shape[0])] for i in range(self.cov2.shape[1])])
      
        
    def get_request(self,req):
        r = self.session.get(self.url + "/" + req, verify = False) 
        return json.loads(r.text)
    def get_assetsids(self):
        r = self.session.get(self.url + "/asset" , verify = False)
        dic = json.loads(r.text)
        N = len(dic)
        ass_ids = np.zeros(N)
        for i in range(N):
            ass_ids[i] = int(dic[i]["ASSET_DATABASE_ID"]["value"])
        self.ids = ass_ids.astype(int)
        self.write_file(self.ids, "my_ids.txt")
        return self.ids
    def get_asset_vecrs(self, ids, jump = 1, allinone = False):
        start = datetime.datetime(2012,1,1)
        end = datetime.datetime(2017,6,1)
        months = (end.year - start.year) * 12 + (end.month - start.month)
        asset_vecr = []
        asset_vecs =[]
        
        if not(allinone):
            end =  start + relativedelta(months=+jump)
        else:
            months = 1

        
        for i in range(months):
            obj2 = {'ratio':[21,20],'asset':[int(ids[i]) for i in range(len(ids))],'start_date':start.isoformat(),'end_date':end.isoformat(),'frequency':None}
            obj = json.dumps(obj2)
            r = self.session.post(self.url + "/ratio/invoke", data = obj, verify = False)
            dic = json.loads(r.text)
            asset_vecr.append([float((dic[str(ids[i])]['21']['value']).replace(",",'.')) for i in range(len(ids))])
            asset_vecs.append([float((dic[str(ids[i])]['20']['value']).replace(",",'.')) for i in range(len(ids))])
            
            start = start + relativedelta(months=+jump)
            end = end + relativedelta(months=+jump)
        asset_vecr = np.array(asset_vecr)
        asset_vecs = np.array(asset_vecs)
        if not(allinone):
            self.write_file(asset_vecr.T, 'my_assets65.txt')
            self.write_file(asset_vecs.T, 'my_sharpes65.txt')
        else:
            self.write_file(asset_vecr, 'my_assets.txt')
            self.write_file(asset_vecs, 'my_sharpes.txt')
            
        return asset_vecr, asset_vecs
    def write_file(self, data, name):
        f1 = open(name, "w")
        if (len(data.shape) == 2):
            for i in range(data.shape[0]):
                for j in range(data.shape[1] - 1):
                    f1.write(str(data[i][j]) + ' ')
                f1.write(str(data[i,data.shape[1] - 1]) + '\n')
        else:
            for i in range(data.shape[0] - 1):
                    f1.write(str(data[i]) + ' ')
            f1.write(str(data[data.shape[0] - 1]) + '\n')
        f1.close()
    def read_file(self, name, prx_cur = False):
        f1 = open(name, "r")
        data = f1.readlines()

        f1.close()
        if (len(data) == 1):
            return np.array(data[0].split()).astype(np.float32)
        else:
            if(prx_cur):
                data= np.array([np.array(data[i].split()) for i in range(len(data))])
                return np.array([data[:,0].astype(np.float64), data[:,1]])
            else:
                return np.array([np.array(data[i].split()).astype(np.float64) for i in range(len(data))])
    
    
        
    def corr_matrix(self):
        ids = self.ids
        start = datetime.datetime(2012,1,1)
        end = datetime.datetime(2017,6,1)
        asset_corr = []
        for j in range(len(ids)):
            obj2 = {'ratio':[19],'asset':[int(ids[i]) for i in range(len(ids))],'benchmark':int(ids[j]),'start_date':start.isoformat(),'end_date':end.isoformat(),'frequency':None}
            obj = json.dumps(obj2)
            r = self.session.post(self.url + "/ratio/invoke", data = obj, verify = False)
            dic = json.loads(r.text)
            asset_corr.append([float((dic[str(ids[i])]['19']['value']).replace(",",'.')) for i in range(len(ids))])
            self.corr_mat = np.array(asset_corr)
            self.write_file(self.corr_mat, 'my_corr.txt')
        return self.corr_mat
    def get_variances(self):
        start = datetime.datetime(2012,1,1)
        end = datetime.datetime(2017,6,1)
        ids = self.ids
        var = []
        obj2 = {'ratio':[18],'asset':[int(ids[i]) for i in range(len(ids))],'start_date':start.isoformat(),'end_date':end.isoformat(),'frequency':None}
        obj = json.dumps(obj2)
        r = self.session.post(self.url + "/ratio/invoke", data = obj, verify = False)
        dic = json.loads(r.text)
        var = [float((dic[str(ids[i])]['18']['value']).replace(",",'.'))**2 for i in range(len(ids))]
        self.var = np.array(var)
        self.write_file(self.var, 'my_vars.txt')
        return self.var
    def get_ass_curr(self):
        r = self.get_request("asset")
        prices = []
        currencys = []
        for i in range(len(r)):
            a = r[i]['LAST_CLOSE_VALUE']['value']
            l = np.array(re.split(" |,", a))
            price = [l[i]  for i in range(len(l)) if (l[i].isnumeric())]
            currency  = r[i]['CURRENCY']['value']
            if(len(price) == 2):
                price = float(price[0] + '.' + price[1])
            else :
                price = float(price[0])
            prices.append(price)
            currencys.append(currency)
        self.prx_cur = np.array([np.array(prices), np.array(currencys)]).T
        self.write_file(self.prx_cur,'my_prx_curr.txt')
        return self.prx_cur
    
    def get_cov(self):
        ids = self.ids
        if (self.ids == []):
            self.ids = self.get_assetsids()
        if (self.corr_mat == []):
            self.corr_mat = self.corr_matrix(self, self.ids)
        self.cov = np.array([[self.corr_mat[i][j] / np.sqrt(self.var[i] * self.var[j]) for i in range(self.corr_mat.shape[0])]for j in range(self.corr_mat.shape[1]) ])
        self.write_file(self.cov, 'my_cov.txt')
        return self.cov

    def get_portfoliosh(self, ids, Ret_J = False, Sig_J = True):
        assert(len(ids) ==20), 'the lentgh of indices should be exactly 20'
        w = self.get_weights(ids, Ret_J = Ret_J, Sig_J = Sig_J) 
        Sigma = []
        RT = []
        if not(Ret_J):
            returns = self.assets65[ids]
            RT = np.mean(returns, axis=1).reshape(20,1)
        else :
            RT = self.assets[ids].reshape(20,1)
        if (Sig_J):
            Sigma = self.cov[ids][:,ids]
        else :
            Sigma = self.cov2[ids][:,ids]
        
        rp =  RT.T.dot(w)
        sigmap = np.sqrt(w.T.dot(Sigma.dot(w)))
        sharp = rp/sigmap
        return np.asscalar(sharp)
    def get_asset_quant(self, prx, cur, ammount):
        
        c = CurrencyRates()
        pr = float(prx)
        if (cur == 'EUR'):
            return float(ammount/ pr)
        else:
            val = c.convert(cur,'EUR',pr)
            return float(ammount/val)
    def get_assets_quant(self, ids, w, big_amount):
        assert(len(ids) == len(w)), 'weights and ids are not the same length'
        pc = self.prx_cur
        
        assets_quant = np.array([self.get_asset_quant(pc[i][0], pc[i][1] ,np.asscalar(big_amount * w[i])) for i in range(len(ids))])
        return assets_quant
    def get_weights(self, ids, Ret_J=False, Sig_J= True):
        assert(len(ids) == 20), 'the lentgh of indices should be exactly 20' 
        Sigma = []
        RT = []
        if not(Ret_J):
            returns = self.assets65[ids]
            RT = np.mean(returns, axis=1).reshape(20,1).astype(np.double)
        else :
            RT = self.assets[ids].reshape(20,1).astype(np.double)
        if (Sig_J):
            Sigma = self.cov[ids][:,ids]
        else :
            Sigma = self.cov2[ids][:,ids]
            
        N = 20
        P = matrix(Sigma)
        q = matrix(-2 * RT,tc='d')
        G = matrix(np.vstack((np.identity(N), -np.identity(N))),tc='d')

        h = matrix(np.hstack((0.1 * np.ones(N), np.ones(N) * -0.01)), tc='d')
        A = matrix(np.ones(N), (1,N)) 
        b = matrix([1.0])
        sol = solvers.qp(P, q, G, h, A, b)
        W = sol['x']
        return np.array(W)
    def  dic_ass(self, ids, quants):
        val = [{'asset': {'asset': int(ids[i]), 'quantity': int(quants[i])}} for i in range(len(ids))]
        return val
    def post_portfolio(self, ids, w, amount = 1e7):
        quants = rm.get_assets_quant(ids,w, amount)
        dic_ass = self.dic_ass(ids, quants)
        dic = {'currency': {'code': 'EUR'},'label': 'PORTFOLIO_USER5','type': 'front','values': {'2012-01-01': dic_ass}}
        obj = json.dumps(dic)
        r = self.session.put(self.url + "/portfolio/"+str(self.port_id)+"/dyn_amount_compo", data = obj, verify = False)
        print("end posting")
        return r

In [7]:
#rm.get_request("asset?columns=ASSET_DATABASE_ID&columns=LABEL&columns=TYPE&TYPE=PORTFOLIO")
#rm.get_request("portfolio/573/dyn_amount_compo")
URL = 'https://dolphin.jump-technology.com:3389/api/v1'
user = 'epita_user_5'
pwd = 'dolphin23235'
rm = RequestManadger(URL,user, pwd)



/usr/lib/python3/dist-packages/urllib3/connectionpool.py:794: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)


## 1 ) Trivial Classificaion by best sharps

In [8]:
f1 = open("my_sharpes.txt","r")
sharpes = np.array(f1.readline().split()).astype(np.float32)
f1.close()
idx = np.argsort(sharpes)[::-1]
ids = idx[:20]
sh1 = rm.get_portfoliosh(ids,Ret_J=True, Sig_J=True)
sh2 = rm.get_portfoliosh(ids, Ret_J=True, Sig_J=False)
sh3 = rm.get_portfoliosh(ids, Ret_J=False, Sig_J=True)
sh4 = rm.get_portfoliosh(ids, Ret_J=False, Sig_J=False)
print("Use Jump Return + Jump Sigma  :", sh1)
print("Use Jump Return + 65 Sigma  :", sh2)
print("Use 65 Return +  Jump Sigma :", sh3)
print("Use 65 Return + 65 Sigma :", sh4)
print("Sum of sharpes :", sh1 + sh2 + sh3 + sh4)


Use Jump Return + Jump Sigma  : 0.20161985138146446
Use Jump Return + 65 Sigma  : 80.65770457071
Use 65 Return +  Jump Sigma : 0.001624921225077248
Use 65 Return + 65 Sigma : 0.6535830034613018
Sum of sharpes : 81.51453234677784


# 2) Sharpe of sharpe classification

In [9]:
f2 = open("my_sharpes65.txt", "r")
data2 = f2.readlines() 
sharpes = np.array([np.array(data2[i].split()).astype(np.float64) for i in range(len(data2))])
f2.close()
eps = 1e-5
#rfs = (sharpes.max() + sharpes.min() ) / 2
rfs = sharpes.mean()
resharpes = np.array([(sharpes[i].mean() - rfs)/np.var(sharpes[i]) for i in range(sharpes.shape[0])])
idx2 = np.argsort(resharpes)[::-1]
#print(idx2)
ids = idx2[:20]
sh1 = rm.get_portfoliosh(ids,Ret_J=True, Sig_J=True)
sh2 = rm.get_portfoliosh(ids, Ret_J=True, Sig_J=False)
sh3 = rm.get_portfoliosh(ids, Ret_J=False, Sig_J=True)
sh4 = rm.get_portfoliosh(ids, Ret_J=False, Sig_J=False)
print("Use Jump Return + Jump Sigma  :", sh1)
print("Use Jump Return + 65 Sigma  :", sh2)
print("Use 65 Return +  Jump Sigma :", sh3)
print("Use 65 Return + 65 Sigma :", sh4)
print("Sum of sharpes :", sh1 + sh2 + sh3 + sh4)

Use Jump Return + Jump Sigma  : 4.412822786111729
Use Jump Return + 65 Sigma  : 34.20628368897921
Use 65 Return +  Jump Sigma : 0.08297205114393717
Use 65 Return + 65 Sigma : 0.34283499788777994
Sum of sharpes : 39.04491352412266


In [77]:
w = rm.get_weights(ids,Ret_J=True, Sig_J=True)

In [78]:
rm.post_portfolio(ids, w)

end posting


/usr/lib/python3/dist-packages/urllib3/connectionpool.py:794: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)


<Response [404]>

# 3) the most decorelated sharpe classification

In [49]:
#we reduce the assets by dividing by sigma as the correlation matrix would be exactly as covariance matrix because
#we want to use the property cov(x+y, z) = cov(x,z) + cov(y, z)
assets_ret_reduced = np.array([assets_ret[i]/np.sqrt(np.var(assets_ret[i])) for i in range(assets_ret.shape[0])])
idx3 = idx[:100]
b = assets_ret_reduced[idx3]
n = 20
btmp = assets_ret[0]
best_assets = [btmp]
# Sequence initialization
Xn0 = btmp
Xn1 = btmp
cor1 = np.corrcoef(b)
Zn0 = assets_ret_reduced[cor1[0].argmin()]
Zn1 = Zn0
b1 = np.array(b)
b2 = np.array(b)
index = 0
for i in range(n - 1):
    #print(b.shape)
    b1 = np.delete(b1, 0, axis = 0)
    b1 = b1.insert(b1,0, Xn1, axis = 0)
    
    b2 = np.delete(best_assets[-1])
    cor1 = np.corrcoef(b1)
    #print(cor)
    i = np.argmin(np.abs(cor[0]))
    assert(btmp.shape == b[i].shape),'error shape'
    btmp = btmp + b[i]
    #print(b.shape)
    #print(btmp.shape,"btmp")
    b = np.delete(b, 0, axis = 0)
    b = np.insert(b,0,btmp, axis = 0)

best_ever = b[indexs]


TypeError: delete() missing 1 required positional argument: 'obj'